Copyright (c) Microsoft Corporation. All rights reserved.  
Licensed under the MIT License.

![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/NotebookVM/how-to-use-azureml/machine-learning-pipelines/intro-to-pipelines/aml-pipelines-with-data-dependency-steps.png)

# Azure Machine Learning Pipelines with Data Dependency
In this notebook, we will see how we can build a pipeline with implicit data dependency.

### Azure Machine Learning and Pipeline SDK-specific Imports

In [15]:
import azureml.core
import azureml.dataprep
from azureml.core import Workspace, Experiment, Datastore, Dataset
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.widgets import RunDetails
from azureml.data import TabularDataset
from azureml.data.data_reference import DataReference
from azureml.pipeline.core import Pipeline, PipelineData
from azureml.pipeline.steps import PythonScriptStep

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.0.85


### Initialize Workspace and Retrieve a Compute Target

In [16]:
ws = Workspace.from_config()
print("== Workspace:")
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

# Default datastore (Azure blob storage)
# def_blob_store = ws.get_default_datastore()
blob_store = Datastore(ws, "workspaceblobstore")
print("== Datastore: {}".format(blob_store.name))

# list compute targets
print("== Compute targets:")
for ct in ws.compute_targets:
    print("  " + ct)
    
# Retrieve a compute target    
from azureml.core.compute_target import ComputeTargetException
aml_compute_target = "agd-training-cpu"
try:
    aml_compute = AmlCompute(ws, aml_compute_target)
    print("== AML compute target attached: " + aml_compute_target)
except ComputeTargetException:
    print("== AML compute target not found: " + aml_compute_target)

== Workspace:
agd-ml-demo
azure-ml-workshop
westus2
c5ec24ce-9c5f-4da2-bf12-9ca8e9758d60
== Datastore: workspaceblobstore
== Compute targets:
  agd-training-cpu
== AML compute target attached: agd-training-cpu


### Create Compute Configuration

This step uses a docker image, use a [**RunConfiguration**](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.runconfiguration?view=azure-ml-py) to specify these requirements and use when creating the PythonScriptStep. 

In [17]:
from azureml.core.runconfig import RunConfiguration
from azureml.core.runconfig import DEFAULT_CPU_IMAGE
from azureml.core.conda_dependencies import CondaDependencies

# create a new runconfig object
run_config = RunConfiguration()

# enable Docker 
run_config.environment.docker.enabled = True

# set Docker base image to the default CPU-based image
run_config.environment.docker.base_image = DEFAULT_CPU_IMAGE

# use conda_dependencies.yml to create a conda environment in the Docker image for execution
run_config.environment.python.user_managed_dependencies = False

# specify dependencies
#run_config.environment.python.conda_dependencies = CondaDependencies.create(
#    pip_packages=['azureml-dataprep[fuse,pandas]'])
    #    conda_packages=['pandas'],
#    pip_packages=['azureml-sdk', 'azureml-dataprep', 'azureml-train-automl'], 
#    pin_sdk_version=False)
run_config.environment.python.conda_dependencies = CondaDependencies(
    conda_dependencies_file_path='data-prep-pipeline.yml')

#
print("== Run Configuration created")

== Run Configuration created


## Building Pipeline Steps with Inputs and Outputs
As mentioned earlier, a step in the pipeline can take data as input. This data can be a data source that lives in one of the accessible data locations, or intermediate data produced by a previous step in the pipeline.

### Datasources as Datasets

In [18]:
# hourly time series
h_time_series_1_ds = Dataset.get_by_name(ws,"h_time_series_1")
h_time_series_2_ds = Dataset.get_by_name(ws,"h_time_series_2")
h_time_series_3_ds = Dataset.get_by_name(ws,"h_time_series_3")
# daily time series
d_time_series_1_dr = DataReference(datastore=blob_store,
                                   data_reference_name="d_time_series_1",
                                   path_on_datastore="datasets/time-series/X1.csv")
d_time_series_2_dr = DataReference(datastore=blob_store,
                                   data_reference_name="d_time_series_2",
                                   path_on_datastore="datasets/time-series/X2.csv")

print("== Datasets metadata retrieved")

== Datasets metadata retrieved


### Intermediate/Output Data
Intermediate data (or output of a Step) is represented by **[PipelineData](https://docs.microsoft.com/en-us/python/api/azureml-pipeline-core/azureml.pipeline.core.pipelinedata?view=azure-ml-py)** object. PipelineData can be produced by one step and consumed in another step by providing the PipelineData object as an output of one step and the input of one or more steps.

In [19]:
# Define intermediate data using PipelineData
# hourly series that need to have timestampts generated from columns and pivoted if necessary
h_time_series_1_pivot_pd = PipelineData("h_time_series_1_pivot",datastore=blob_store)
h_time_series_2_pivot_pd = PipelineData("h_time_series_2_pivot",datastore=blob_store)
h_time_series_3_pivot_pd = PipelineData("h_time_series_3_pivot",datastore=blob_store)
print("== Intermediate PipelineData references created")

== Intermediate PipelineData references created


### Pipelines steps using datasources and intermediate data
Machine learning pipelines can have many steps and these steps could use or reuse datasources and intermediate data. Here's how we construct such a pipeline:

In [20]:
# The best practice is to use separate folders for scripts and its dependent files
# for each step and specify that folder as the source_directory for the step.
# This helps reduce the size of the snapshot created for the step (only the specific folder is snapshotted).
# Since changes in any files in the source_directory would trigger a re-upload of the snapshot, this helps
# keep the reuse of the step when there are no changes in the source_directory of the step.
source_directory_pivot = 'src/pivot'
source_directory_join = 'src/join'
source_directory_groupby_aggregate = 'src/groupby-aggregate'

In [21]:
# step4 consumes the datasource (Datareference) in the previous step
# and produces processed_data1
h_time_series_1_pivot_step = PythonScriptStep(
    script_name="pivot.py", 
    arguments=["--date_column","MYDATE",
               "--hour_column","HOUR",
               "--datetime_column_name","DATETIME",
               "--pivot_columns","NODE_ID",
               "--value_column","MW",
               "--output", h_time_series_1_pivot_pd],
    inputs=[h_time_series_1_ds.as_named_input("time_series")],
    outputs=[h_time_series_1_pivot_pd],
    compute_target=aml_compute, 
    source_directory=source_directory_pivot,
    runconfig=run_config
)
print("== PythonScriptStep time_series_1_pivot_step created")

h_time_series_2_pivot_step = PythonScriptStep(
    script_name="pivot.py", 
    arguments=["--date_column","MYDATE",
               "--hour_column","HOUR",
               "--datetime_column_name","DATETIME",
               "--pivot_columns","NODE_ID",
               "--value_column","MW",
               "--output", h_time_series_2_pivot_pd],
    inputs=[h_time_series_2_ds.as_named_input("time_series")],
    outputs=[h_time_series_2_pivot_pd],
    compute_target=aml_compute, 
    source_directory=source_directory_pivot,
    runconfig=run_config
)
print("== PythonScriptStep time_series_2_pivot_step created")

h_time_series_3_pivot_step = PythonScriptStep(
    script_name="pivot.py", 
    arguments=["--date_column","DATE",
               "--hour_column","HE",
               "--datetime_column_name","DATETIME",
               "--pivot_columns","",
               "--value_column","CLOAD",
               "--output", h_time_series_3_pivot_pd],
    inputs=[h_time_series_3_ds.as_named_input("time_series")],
    outputs=[h_time_series_3_pivot_pd],
    compute_target=aml_compute, 
    source_directory=source_directory_pivot,
    runconfig=run_config
)
print("== PythonScriptStep time_series_3_pivot_step created")

== PythonScriptStep time_series_1_pivot_step created
== PythonScriptStep time_series_2_pivot_step created
== PythonScriptStep time_series_3_pivot_step created


#### Define a Step that consumes intermediate data and produces intermediate data
In this step, we define a step that consumes an intermediate data and produces intermediate data.

**Open `join.py` in the local machine and examine the arguments, inputs, and outputs for the script. That will give you a good sense of why the script argument names used below are important.** 

In [22]:
# Joining all HOURLY pivoted time-series
h_time_series_joined_ds = PipelineData('h_time_series_joined', datastore=blob_store).as_dataset()
h_time_series_joined_ds = h_time_series_joined_ds.register(name='h_time_series_joined', create_new_version=True)

# Join Step
h_join_step = PythonScriptStep(
    script_name="join.py",
    arguments=["--join_column", "DATETIME",
               "--input_1", h_time_series_1_pivot_pd,
               "--input_2", h_time_series_2_pivot_pd,
               "--input_3", h_time_series_3_pivot_pd,
               "--output", h_time_series_joined_ds],
    inputs=[h_time_series_1_pivot_pd,
            h_time_series_2_pivot_pd,
            h_time_series_3_pivot_pd],
    outputs=[h_time_series_joined_ds],
    compute_target=aml_compute, 
    source_directory=source_directory_join)

print("== PythonScriptStep h_join_step created")

== PythonScriptStep h_join_step created


In [23]:
# Joining all DAILY time-series
d_time_series_joined_ds = PipelineData('d_time_series_joined', datastore=blob_store).as_dataset()
d_time_series_joined_ds = d_time_series_joined_ds.register(name='d_time_series_joined', create_new_version=True)

# Join Step
d_join_step = PythonScriptStep(
    script_name="join.py",
    arguments=["--join_column", "RDATE",
               "--input_1", d_time_series_1_dr,
               "--input_2", d_time_series_2_dr,
               "--output", d_time_series_joined_ds],
    inputs=[d_time_series_1_dr,
            d_time_series_2_dr],
    outputs=[d_time_series_joined_ds],
    compute_target=aml_compute,
    source_directory=source_directory_join)

print("== PythonScriptStep d_join_step created")

== PythonScriptStep d_join_step created


In [24]:
# Generate stats for HOURLY joined series GROUP BY DAILY
h_time_series_joined_groupby_aggregated_ds = PipelineData('h_time_series_joined_groupby_aggregated', datastore=blob_store).as_dataset()
h_time_series_joined_groupby_aggregated_ds = h_time_series_joined_groupby_aggregated_ds.register(name='h_time_series_joined_groupby_aggregated', create_new_version=True)

# groupby-aggregate step
groupby_aggregate_step = PythonScriptStep(
    script_name="groupby-aggregate.py",
    arguments=["--datetime_column", "DATETIME",
               "--date_column_name","RDATE",
               "--input_ds", h_time_series_joined_ds,
               "--output_ds", h_time_series_joined_groupby_aggregated_ds],
    inputs=[h_time_series_joined_ds],
    outputs=[h_time_series_joined_groupby_aggregated_ds],
    compute_target=aml_compute,
    source_directory=source_directory_groupby_aggregate)

print("== PythonScriptStep groupby_aggregate_step created")

== PythonScriptStep groupby_aggregate_step created


### Build the pipeline and submit an Experiment run

In [25]:
pipeline = Pipeline(workspace=ws, steps=[h_join_step,d_join_step,groupby_aggregate_step])
print ("== Pipeline is built")

== Pipeline is built


In [26]:
pipeline_run = Experiment(ws, 'use-case-1-data-prep').submit(pipeline)
print("== Pipeline is submitted for execution")

Created step join.py [aa7b6a37][237a17df-f481-4c9b-a503-6e573d32b214], (This step is eligible to reuse a previous run's output)
Created step pivot.py [445b729a][bfd9831a-22db-49f9-b273-5d944f34e29a], (This step is eligible to reuse a previous run's output)
Created step pivot.py [866692d3][1855b634-0083-4a72-8a18-eaeda07375d1], (This step is eligible to reuse a previous run's output)
Created step pivot.py [74f2e610][2a23178d-0e17-4791-9475-fba2a1f33bcb], (This step is eligible to reuse a previous run's output)
Created step join.py [c5ae7802][7842a40d-bfd6-455c-9dd7-7667a5d80aec], (This step is eligible to reuse a previous run's output)
Created step groupby-aggregate.py [069ed222][1b3761bf-34b3-4c61-9df4-5b06df0cd96a], (This step is eligible to reuse a previous run's output)
Using data reference d_time_series_1 for StepId [dfde5f9b][466adead-675c-4485-b1f6-3a08282a68dd], (Consumers of this data are eligible to reuse prior runs.)
Using data reference d_time_series_2 for StepId [c82deab7][

In [27]:
RunDetails(pipeline_run).show()

_PipelineWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', …

### Wait for pipeline run to complete

In [28]:
pipeline_run.wait_for_completion(show_output=True)

PipelineRunId: 65c5d560-c39d-4810-959b-5d9ead59e1a4
Link to Portal: https://ml.azure.com/experiments/use-case-1-data-prep/runs/65c5d560-c39d-4810-959b-5d9ead59e1a4?wsid=/subscriptions/c5ec24ce-9c5f-4da2-bf12-9ca8e9758d60/resourcegroups/azure-ml-workshop/workspaces/agd-ml-demo
PipelineRun Status: NotStarted
PipelineRun Status: Running


StepRunId: f82fb7d4-42a8-4b21-8149-ef3a6fbeceb1
Link to Portal: https://ml.azure.com/experiments/use-case-1-data-prep/runs/f82fb7d4-42a8-4b21-8149-ef3a6fbeceb1?wsid=/subscriptions/c5ec24ce-9c5f-4da2-bf12-9ca8e9758d60/resourcegroups/azure-ml-workshop/workspaces/agd-ml-demo

StepRun(join.py) Execution Summary
StepRun( join.py ) Status: Finished
{'runId': 'f82fb7d4-42a8-4b21-8149-ef3a6fbeceb1', 'target': 'agd-training-cpu', 'status': 'Completed', 'startTimeUtc': '2020-04-02T17:30:17.304611Z', 'endTimeUtc': '2020-04-02T17:30:17.389038Z', 'properties': {'azureml.reusedrunid': '52dc4e46-6df5-40fe-aef3-ec6c04848ca4', 'azureml.reusednodeid': '334ea4e5', 'azureml

StepRun( pivot.py ) Status: Running

Streaming azureml-logs/55_azureml-execution-tvmps_8d61c9874ff3d5b1f6b6b1c4382e01d968590c71c842d13c88a40d456aa5093e_d.txt
2020-04-02T17:31:03Z Starting output-watcher...
2020-04-02T17:31:03Z IsDedicatedCompute == True, won't poll for Low Pri Preemption
Login Succeeded
Using default tag: latest
latest: Pulling from azureml/azureml_fbecfa6255cba4c1cc15571eceadee24
Digest: sha256:594ef1dded733ed51cf75ec7f5bd4227657d6aec1dba0d03763c299995ee1b79
Status: Image is up to date for agdmldemoaf0903d3.azurecr.io/azureml/azureml_fbecfa6255cba4c1cc15571eceadee24:latest
7108642d118492598d1dae4ec48dd165c774308ff811b96412d1e406292aa99a
2020/04/02 17:31:05 Version: 3.0.01172.0001 Branch: master Commit: d33e301a
2020/04/02 17:31:06 /dev/infiniband/uverbs0 found (implying presence of InfiniBand)?: false
2020/04/02 17:31:06 sshd runtime has already been installed in the container

Streaming azureml-logs/65_job_prep-tvmps_8d61c9874ff3d5b1f6b6b1c4382e01d968590c71c842d13c88

ActivityFailedException: ActivityFailedException:
	Message: Activity Failed:
{
    "error": {
        "code": "UserError",
        "message": "User program failed with TypeError: can only concatenate str (not \"datetime.timedelta\") to str",
        "detailsUri": "https://aka.ms/azureml-known-errors",
        "details": [],
        "debugInfo": {
            "type": "TypeError",
            "message": "can only concatenate str (not \"datetime.timedelta\") to str",
            "stackTrace": "  File \"/mnt/batch/tasks/shared/LS_root/jobs/agd-ml-demo/azureml/f630b51f-b7cc-4e6b-ad93-a50404397b31/mounts/workspaceblobstore/azureml/f630b51f-b7cc-4e6b-ad93-a50404397b31/azureml-setup/context_manager_injector.py\", line 127, in execute_with_context\n    runpy.run_path(sys.argv[0], globals(), run_name=\"__main__\")\n  File \"/azureml-envs/azureml_ed39ba8e8af4352edb337d7a448246d0/lib/python3.7/runpy.py\", line 263, in run_path\n    pkg_name=pkg_name, script_name=fname)\n  File \"/azureml-envs/azur

### See Outputs

See where outputs of each pipeline step are located on your datastore.

***Wait for pipeline run to complete, to make sure all the outputs are ready***

In [ ]:
# Get Steps
for step in pipeline_run.get_steps():
    print("== Outputs of step " + step.name)
    
    # Get a dictionary of StepRunOutputs with the output name as the key 
    output_dict = step.get_outputs()
    
    for name, output in output_dict.items():
        output_reference = output.get_port_data_reference() # Get output port data reference
        print("\tname: " + name)
        print("\tdatastore: " + output_reference.datastore_name)
        print("\tpath on datastore: " + output_reference.path_on_datastore)

# Next: Publishing the Pipeline and calling it from the REST endpoint
See this [notebook](https://aka.ms/pl-pub-rep) to understand how the pipeline is published and you can call the REST endpoint to run the pipeline.